In [ ]:
from scipy.integrate import odeint
from scipy.integrate import quad, solve_ivp
from scipy import integrate
import numpy as np
import ZebraLib as zb
import pandas as pd
import matplotlib.pyplot as plt
import random
from tqdm import tqdm

In [ ]:
Zb = zb.Airplane()

Zb2 = zb.Airplane(name='2022', S=1.06, b=2.05, 
                  c=0.529, Af=0.605, CLmax=1.474, CLn=1.021)

Zb3 = zb.Airplane(name='teste', S=1.06, b=2.05, Tc=(-0.001, -0.225, 33.34),
                  c=0.529, Af=0.605, CLmax=1.474, CLn=1.021)

df = pd.concat([Zb.info(), Zb2.info()], axis=1)
df

In [ ]:
def resid(x, args):
    rho = args
    Zb = zb.Airplane(name='2022', S=1.06, b=2.05, 
                  c=0.529, Af=0.605, CLmax=1.474, CLn=1.021, Load=x)
    return 50 - (Zb.takeOff_Distance_AproxMiranda())
        
res = zb.robustNewton(resid, (6., 6.), args=0)
res

In [ ]:
def resid(x, args):
    rho = args
    Zb = zb.Airplane(name='2022', S=1.06, b=2.05, 
                  c=0.529, Af=0.605, CLmax=1.474, CLn=1.021, Load=x)
    return 50 - (Zb.takeOff_Distance_EDO())
        
res = zb.robustNewton(resid, (6., 6.), args=0)
res